# Quantum Neuron Born Machine
**Quantum Neuron Born Machine** (QNBM) is a model that adds non-linear activations via a neural network structure onto the standard Born Machine framework. The QNBM utilizes a previously introduced Quantum Neuron subroutine, which is a repeat-until-success circuit with mid-circuit measurements and classical control. The QNBM has been shown to achieve an almost 3x smaller error rate than a linear Quantum Circuit Born Machine (QCBM) with a similar number of tunable parameters. This suggests that non-linearity is a useful resource in quantum generative models, and the QNBM is a new model with good generative performance and potential for quantum advantage.


## Connect to the Azure Quantum workspace

In [1]:
import qsharp
import qsharp.azure

targets = qsharp.azure.connect(
            resourceId = "",
            location = "")


print("This workspace's targets:")
for target in targets:
    print("-", target.id)

Preparing Q# environment...


Connecting to Azure Quantum...

Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace QuantumDemo in location westcentralus.
This workspace's targets:
- ionq.qpu
- ionq.qpu.aria-1
- ionq.simulator
- microsoft.estimator
- quantinuum.qpu.h1-1
- quantinuum.sim.h1-1sc
- quantinuum.qpu.h1-2
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-2e
- rigetti.sim.qvm
- rigetti.qpu.aspen-m-2
- rigetti.qpu.aspen-m-3


In [2]:
import numpy as np
import pandas as pd 
import typing
import math


def counts_to_prob(self, samples: typing.Dict): #Turns a dictionary of sample counts into probabilities.
    total_shots = np.sum(list(samples.values()))
    for bitstring, count in samples.items(): 
        samples[bitstring] = count / total_shots 
    return samples 

def dec2bin(number: int, length: int) -> typing.List[int]: #Turns a decimal number into a binary bitstring.
    bit_str = bin(number)
    bit_str = bit_str[2 : len(bit_str)]  
    bit_string = [int(x) for x in list(bit_str)]
    if len(bit_string) < length:
        len_zeros = length - len(bit_string)
        bit_string = [int(x) for x in list(np.zeros(len_zeros))] + bit_string
    return bit_string

def comb(n: int, k: int)-> int: #Combinatorial method for n choose k.
    return math.factorial(n) // math.factorial(k) // math.factorial(n - k)

In [3]:
from qiskit import Aer,QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit.providers.ibmq.runtime import UserMessenger, ProgramBackend
from qiskit.algorithms.optimizers import NELDER_MEAD, ADAM, GradientDescent
from qiskit.providers.aer import AerSimulator
from qiskit import transpile
import numpy as np
from numpy import pi
import math

class QuantumNeuralNetwork:

    def __init__(self, width_list, k):
        self.structure = width_list
        self.depth = len(width_list)
        self.edge_no = self.EdgeNo() 
        self.neuron_no = sum(width_list) 
        self.ancilla_qubit_no = k 
        self.cbits_per_neuron = 2**k - 1
        self.ancilla_bit_no = (self.neuron_no - self.structure[0]) * (2**k - 1)
        self.answer_bit_no = self.structure[self.depth - 1]
        self.qubit_no = self.neuron_no + self.ancilla_qubit_no 
        self.bit_no = self.ancilla_bit_no + self.answer_bit_no

    def EdgeNo(self): #Defines the number of edges in the model 
        edgeno = 0
        for i in range(len(self.structure)-1):
            edgeno += self.structure[i] * self.structure[i+1]
        return edgeno

    def Normalise(self, weights_unnormalised, biases_unnormalised): #Normalizes the weights and biases of the model 
        weight_counter = 0
        bias_counter = 0
        weights = np.copy(weights_unnormalised)
        biases = np.copy(biases_unnormalised)
        for i in range(self.depth-1):
            current_layer = self.structure[i]
            next_layer = self.structure[i+1]
            normalisation = pi/4 / (current_layer + 1)
            no_of_weights = current_layer * next_layer
            no_of_biases = next_layer
            weights[weight_counter : weight_counter + no_of_weights] = weights[weight_counter : weight_counter + no_of_weights] * normalisation
            biases[bias_counter : bias_counter + no_of_biases] = biases[bias_counter : bias_counter + no_of_biases] * normalisation + pi/4
            weight_counter += no_of_weights
            bias_counter += no_of_biases
        return weights, biases
        
    def CreateCircuit(self, weights_unnorm, biases_unnorm, initialisation = 'h'): #Defines the neruon circuit using the weights and biases 

        circ = QuantumCircuit(self.qubit_no, self.bit_no)
        first_layer_size = self.structure[0]
        weights, biases = self.Normalise(weights_unnorm, biases_unnorm)
        print("Normalized weights and biases: ", weights, biases)
        if initialisation == 'h':
            for i in range(self.ancilla_qubit_no, self.ancilla_qubit_no + first_layer_size):
                circ.h(i)
        else:
            circ = circ.compose(initialisation, range(self.ancilla_qubit_no, self.ancilla_qubit_no + first_layer_size))
        qubit_counter = self.ancilla_qubit_no 
        weight_counter = 0
        bias_counter = 0 
        cbit_counter = 0 
        for i in range(self.depth-1):
            circ, qubit_counter, weight_counter, bias_counter, cbit_counter = self.AddLayer(circ, qubit_counter, weight_counter, bias_counter, cbit_counter, i, weights, biases)
        circ.measure(range(self.qubit_no - self.answer_bit_no, self.qubit_no), range(self.ancilla_bit_no, self.bit_no))
        return circ

    def AddLayer(self, circ, qubit_counter, weight_counter, bias_counter, cbit_counter, i, weights, biases): #Adds a neuron layer to the circuit  
        previous_layer_size = self.structure[i]
        next_layer_size = self.structure[i+1]
        previous_layer = range(qubit_counter, qubit_counter + previous_layer_size)
        next_layer = range(qubit_counter + previous_layer_size, qubit_counter + previous_layer_size + next_layer_size) 

        for output_qubit in next_layer:
            relevant_weights = weights[weight_counter : weight_counter + previous_layer_size]
            relevant_bias = biases[bias_counter]
            circ = self.ConnectLayerToNode(circ, cbit_counter, previous_layer, output_qubit, relevant_weights, relevant_bias, self.ancilla_qubit_no)
            cbit_counter += self.cbits_per_neuron
            weight_counter += previous_layer_size
            bias_counter += 1
        qubit_counter += previous_layer_size
        return circ, qubit_counter, weight_counter, bias_counter, cbit_counter

    def ConnectLayerToNode(self, circ, cbit_counter, previous_layer, output_qubit, weights, bias, k): #Connects the previous layer to a neuron in the next layer  
        if k == 1:
            for i, input_qubit_i in enumerate(previous_layer):
                weight_i = weights[i]
                circ.cry(2*weight_i, input_qubit_i, 0)
            circ.ry(2*bias, 0)
            circ.cy(0, output_qubit)
            circ.rz(-pi/2, 0)
            for i, input_qubit_i in enumerate(previous_layer):
                weight_i = weights[i]
                circ.cry(-2*weight_i, input_qubit_i, 0)
            circ.ry(-2*bias, 0)
            circ.measure(0, cbit_counter)
            circ.reset(0)
        else:
            circ = self.ConnectLayerToNode(circ, cbit_counter, previous_layer, k-1, weights, bias, k-1)
            circ.cy(k-1, output_qubit)
            circ.rz(-pi/2, k-1)
            circ = self.ConnectLayerToNode(circ, cbit_counter + 2**(k-1) - 1, previous_layer, k-1, -1*weights, -1*bias, k-1)
            circ.measure(k-1, cbit_counter + 2**k - 2)
            circ.reset(k-1)
        return circ

    def CreateCircuitList(self, weights_list, biases_list): #Keeps track of the weights and biases in the model 
        list = []
        for i in range(len(weights_list)):
            weights = weights_list[i]
            biases = biases_list[i]
            list.append(self.CreateCircuit(weights,biases))
        return list

    def FindProbs(self, counts, epsilon = 0, datatype = 'dict'): #Converts the circuit counts to an approximate probability distribution 
        new_counts = {}
        probs_dict = {}
        sum = 0
        for count in counts:
            if count[self.bit_no - self.ancilla_bit_no:] == '0'*self.ancilla_bit_no:
                new_counts[count[0:self.answer_bit_no]] = counts[count]
                sum += counts[count]
        for new_count in new_counts:
            probs_dict[new_count] = new_counts[new_count]/sum
        if datatype == 'dict':
            return probs_dict
        else:
            if datatype == 'array':
                return self.ProbsDict_to_ProbsArray(probs_dict, epsilon)
            else:
                print('only dict and array are accepted as types')
                return

    def ProbsDict_to_ProbsArray(self, probs_dict,epsilon): #Converts the distribution dictionary to an array 
        keys = list(probs_dict.keys())
        keyno = len(keys)
        bitno = len(keys[0])
        nocounts = 2**bitno - keyno
        beta = epsilon * nocounts / keyno
        probs_array = np.zeros(2**bitno)
        for i in range(2**bitno):
            i_bin = format(i, '0'+str(bitno)+'b')
            if i_bin in probs_dict:
                probs_array[i] = probs_dict[i_bin] - beta
            else:
                probs_array[i] = epsilon
        return probs_array

    def FindCounts(self, counts): #Gets the correct counts of the output neurons 
        new_counts = {}
        for count in counts:
            if count[self.bit_no - self.ancilla_bit_no:] == '0'*self.ancilla_bit_no:
                new_counts[count[0:self.answer_bit_no]] = counts[count]
        return new_counts
    def KL(self, trained_distribution, target_distribution,epsilon):
        target_distribution = self.ProbsDict_to_ProbsArray(target_distribution, epsilon)
        trained_distribution = self.ProbsDict_to_ProbsArray(trained_distribution, epsilon)
        return np.dot(target_distribution, np.log(target_distribution/trained_distribution))
        
def main(backend: ProgramBackend, user_messenger: UserMessenger, target_distribution, structure, k, initial_weights, initial_biases, maxiter, eps, lr, seed, gradient = "finite"): #Runs the QNBM algorithm 
    epsilon = 1e-16 
    weight_no = len(initial_weights)
    bias_no = len(initial_biases)
    initial_params = np.concatenate([initial_weights, initial_biases])
    qnn = QuantumNeuralNetwork(structure, k)

    iteration_list = []
    params_list = []
    KL_list = []
    
    def runqnn(parameters): 
        shots = 102400
        params_list.append(parameters)
        w = parameters[0:weight_no]
        b = parameters[weight_no:]
        circuit = qnn.CreateCircuit(w,b)
        tcirc = transpile(circuit, backend)

        # Execute noisy simulation and get counts
        result = backend.run(tcirc, shots=shots).result()
        counts = result.get_counts()
        
        probs = qnn.FindProbs(counts, epsilon, 'array')
        KL_list.append(KL(probs))
        iteration_list.append(1)
        if len(iteration_list) % len(parameters) == 0: 
            print(KL(probs))
        return KL(probs)    
       
    def KL(trained_distribution):
        return np.dot(target_distribution, np.log(target_distribution/trained_distribution))
    
    var_bounds = [(-1,1)]*(weight_no+bias_no)
    np.random.seed(seed)
    optimizer = ADAM(maxiter = maxiter, eps=eps, lr = lr) 
    if gradient == "finite":
        result = optimizer.minimize(runqnn,
                       initial_params,
                       bounds = var_bounds)
 
    params_list.append(result.x)
    KL_list.append(result.fun)
    return params_list, KL_list

In [4]:
import numpy as np
import pandas as pd 
import typing

def subset_distribution( #Generates a cardinality/hamming training probability distribution in the form of a dictionary
    n_qubits: int, 
    numerality: int
    ) -> typing.Dict: 
    dict = {}
    prob = 1 / comb(n_qubits, numerality)
    for i in range(2**n_qubits):
        i_bin = format(i, '0'+str(n_qubits)+'b')
        counter = 0
        for bit in i_bin:
            if bit == '1':
                counter += 1
        if counter == numerality:
            dict[i_bin] = prob
    return dict

In [5]:
%%qsharp
open Microsoft.Quantum.Intrinsic;     // for H
open Microsoft.Quantum.Measurement;   // for MResetZ
open Microsoft.Quantum.Canon;         // for ApplyToEach
open Microsoft.Quantum.Arrays;        // for ForEach
open Microsoft.Quantum.Core;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;

@EntryPoint()
operation createCircuitQ(): (Result[]){
    //1, 0, 2
    let weights = [ 1.34323741, -1.32426818];
    let biases = [0.07816141, 1.41207129];

    let structure = [1, 2];
    let depth = 2;
    let ancilla_qubit_no = 1;
    let neuron_no = 3; //ask about this
    let first_layer_size = structure[0];//1
    let answer_bit_no = structure[depth - 1]; //1
    let qubit_no = neuron_no + ancilla_qubit_no; //3
    
    let num_output = structure[depth - 1]; //1

    use qs = Qubit[qubit_no];
    let end = (ancilla_qubit_no + first_layer_size) - 1; //1
    for qubit in ancilla_qubit_no .. end {
        H(qs[qubit]);
    }
    //All mutables that are needed
    mutable qubit_counter = ancilla_qubit_no; //1
    mutable weight_counter = 0;
    mutable bias_counter = 0;

    for i in 0 .. depth - 2 {
            set (qubit_counter, weight_counter, bias_counter)
            = addLayer(qs, structure, qubit_counter, weight_counter, bias_counter, i, weights, biases, ancilla_qubit_no);
    }
    return MeasureEachZ(qs[qubit_no - answer_bit_no .. (qubit_no-1)]); //2 .. 2
}

operation addLayer(qs: Qubit[], structure: Int[], qubit_counter: Int, weight_counter: Int, bias_counter: Int, i: Int, weights: Double[], biases: Double[], ancilla_qubit_no: Int): (Int, Int, Int){
        mutable weight_counter_new = weight_counter; //0
        mutable bias_counter_new = bias_counter; //0
        mutable qubit_counter_new = qubit_counter; //1
        let previous_layer_size = structure[i]; //1
        let next_layer_size = structure[i+1]; //1
        mutable counter = 0;
        let previous_layer = qubit_counter..(qubit_counter+previous_layer_size-1); //1...1
        let next_layer = qubit_counter+previous_layer_size..(qubit_counter + previous_layer_size + next_layer_size - 1); //2...2

        for output_qubit in next_layer{
                //int that will store the results of the mid-circuit measurements; forcing type to be Int instead of Result -> initialize to 2
            Message($"output_qubit layer: {output_qubit} ");
            mutable succeeded = false;
            for count in 1 .. 7{ //RUS Circuits iterations (for success) are expected to be less than 7
                if not succeeded{
                    let endPos = weight_counter_new + previous_layer_size - 1; //0
                    let relevant_weights = weights[weight_counter_new .. endPos]; //weights[0]
                    let relevant_bias = biases[bias_counter_new]; //biases[0]
                    let cb = measuringAncilla(qs, previous_layer, output_qubit, relevant_weights, relevant_bias, ancilla_qubit_no);
                    if cb == Zero{
                        set succeeded = true;
                    }
                    else{ //if the mid-measurement for the previous RUS circuit came out to be 1 (RUS did not succeed)... 
                        //Ancilla qubit needs to be reset
                        X(qs[0]);
                        Ry(((PI()*-1.0)/2.0), qs[output_qubit-1]);
                    }
                    //Storing the mid circuit measurement
                    // 2 output neurons
                    // output neuron 1 
                    //1 1 1 1...0 -> 110
                }
            }
            set weight_counter_new = weight_counter_new + previous_layer_size; //1
            set bias_counter_new = bias_counter_new + 1; //1
            set counter = counter + 1; //1
        }
        set qubit_counter_new = qubit_counter_new + previous_layer_size; //2
        return (qubit_counter_new, weight_counter_new, bias_counter_new);
}

operation measuringAncilla(qs: Qubit[], previous_layer: Range, output_qubit: Int, weights: Double[], bias: Double, k: Int): Result{
        connectLayerToNode(qs, previous_layer, output_qubit, weights, bias, k);
        if k == 1{
            let result =  M(qs[0]);
            Reset(qs[0]);
            return result;
        }
        else{
            let result = M(qs[k-1]);
            Reset(qs[k-1]);
            return result;
        }
    
}

function Negated(arr : Double[]) : Double[] {   
        mutable negated = [0.0, size=Length(arr)];    
        for idx in IndexRange(arr) {        
            set negated w/= idx <- -arr[idx];    
        }
        return negated;
}

operation connectLayerToNode(qs: Qubit[], previous_layer: Range, output_qubit: Int, weights: Double[], bias: Double, k: Int): Unit is Ctl { 
        if k == 1{
            for i in previous_layer{  //1..1
                let weight_i = weights[i-1]; //weights[0]
                Controlled Ry([qs[i]], ((weight_i*2.0), qs[0]));
            }
            Ry((bias*2.0), qs[0]);
            Controlled Y([qs[0]], (qs[output_qubit]));
            Rz(((PI()*-1.0)/2.0), qs[0]);
            for i in previous_layer{
                let weight_i = weights[i-1]; 
                Controlled Ry([qs[i]], ((weight_i*-2.0), qs[0]));
            }
            Ry((bias*-2.0), qs[0]);
        }
        else{
            connectLayerToNode(qs, previous_layer, k-1, weights, bias, k-1);
            Controlled Y([qs[k-1]], (qs[output_qubit]));
            Rz(((PI()*-1.0)/2.0), qs[k-1]);  
            connectLayerToNode(qs, previous_layer, k-1, Negated(weights), (-1.0*bias), k-1);
        }
}


In [9]:
#qsharp.azure.target("quantinuum.sim.h1-1sc")
#qsharp.azure.target("quantinuum.sim.h1-1e")
qsharp.azure.target("quantinuum.sim.h1-2e")

qsharp.azure.target_capability("AdaptiveExecution")

# We'll use 10 shots (simulated runs). Timeout is in seconds.
result = qsharp.azure.execute(createCircuitQ, shots=10, jobName="QNBM", timeout=240)

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting createCircuitQ to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: QNBM
   Job ID: bcf6c2bc-1962-4931-b1c1-1597d43cd922
Waiting up to 240 seconds for Azure Quantum job to complete...
[18:23:34] Current job status: Waiting
[18:23:39] Current job status: Waiting
[18:23:44] Current job status: Waiting
[18:23:49] Current job status: Executing
[18:23:54] Current job status: Executing
[18:23:59] Current job status: Executing
[18:24:05] Current job status: Executing
[18:24:10] Current job status: Executing
[18:24:15] Current job status: Executing
[18:24:20] Current job status: Executing
[18:24:25] Current job status: Executing
[18:24:30] Current job status: Executing
[18:24:35] Current job status: Executing
[18:24:40] Current job status: Executing
[18:24:45] Current job status: Executing
[18:24:50] Current job status: Succeeded


In [10]:
result

Result,Frequency,Histogram
"[1, 0]",0.7,
"[0, 1]",0.3,
